In [1]:
import os
import cv2
import time
import json
import carla
import torch
import numpy as np
import torch.nn.functional as F
import torchvision.transforms as transforms
from PIL import Image  
from model.ImitationCNN import ImitationCNN
from process_frame import process_frame

/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kaustubh/miniconda3/envs/carla_15/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "models/bc_model.pth"
IMG_RGB = os.path.join('DAgger/rgb', "DAgger_image")
IMG_SEG = os.path.join('DAgger/seg', "DAgger_image")
JSON_PATH = os.path.join('DAgger', 'DAgger_log')
MODEL_PATH = "models/bc_model.pth"
dagger_log = []

os.makedirs(IMG_RGB, exist_ok=True)
os.makedirs(IMG_SEG, exist_ok=True)
os.makedirs(JSON_PATH, exist_ok=True)

In [3]:
model = ImitationCNN().to(device)
model.load_state_dict(torch.load(MODEL_PATH)['model_state_dict'])
model.eval()

ImitationCNN(
  (conv1): Conv2d(5, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc): Linear(in_features=235520, out_features=512, bias=True)
  (steer_head): Linear(in_features=512, out_features=1, bias=True)
  (throttle_head): Linear(in_features=512, out_features=1, bias=True)
  (brake_head): Linear(in_features=512, out_features=1, bias=True)
)

In [4]:
def pre_process(rgb_image, seg_image, transform):
    rgb_tensor = transforms.ToTensor()(rgb_image)
    seg_image = np.array(seg_image)
    lane_mask = np.all(seg_image == [0, 255, 0], axis=2).astype(np.uint8)
    obs_mask = np.all(seg_image == [255, 0, 0], axis=2).astype(np.uint8)
    
    seg_tensor = torch.tensor(np.stack([lane_mask, obs_mask], axis=0), dtype=torch.float32)
    
    if seg_tensor.shape[1:] != rgb_tensor.shape[1:]:
        seg_tensor = F.interpolate(
            seg_tensor.unsqueeze(0),
            size=rgb_tensor.shape[1:],
            mode='nearest'
        ).squeeze(0)
    
    input_tensor = torch.cat([rgb_tensor, seg_tensor], dim=0)
    
    return input_tensor.unsqueeze(0).cuda()


In [5]:
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
blueprint = world.get_blueprint_library()

In [8]:
vehicle_bp = blueprint.find('vehicle.volkswagen.t2_2021')
camera_bp = blueprint.find('sensor.camera.rgb')

In [26]:
spawn_points = world.get_map().get_spawn_points()
image = None

In [10]:
def image_callback(data):
    global image
    array = np.frombuffer(data.raw_data, dtype=np.uint8)
    array = np.reshape(array, (data.height, data.width, 4))[:, :, :3]
    image = array


In [27]:
for actor in world.get_actors().filter("*sensor*"):
    print(actor)
    actor.destroy()
for actor in world.get_actors().filter("*vehicle*"):
    print(actor)
    actor.destroy()
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

vehicle_length = 4.442184
vehicle_width = 1.774566
vehicle_height = 1.987206

camera_location = carla.Location(
    x=vehicle_length / 2.9 , 
    y=0,
    z=2.2
)

camera_transform = carla.Transform(camera_location)

vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])
camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)

expert_vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[1])
expert_camera_sensor = world.spawn_actor(camera_bp, camera_transform, attach_to=vehicle)


Actor(id=73, type=sensor.camera.rgb)
Actor(id=72, type=vehicle.volkswagen.t2_2021)


In [12]:
camera_sensor.listen(lambda image: image_callback(image))

In [13]:
rgb_image = image
seg_image = process_frame(image)
transform = None

In [ ]:
# Testing Behaviour clonning
with torch.no_grad():
    input_tensor = pre_process(rgb_image, seg_image, transform)
    output = model(input_tensor)
    print(output)
    steer, throttle, brake = output[0].cpu().numpy().tolist()
    print(steer, throttle, brake)

    vehicle.apply_control(carla.VehicleControl(
        throttle=throttle,
        steer=steer,
        brake=0,
        hand_brake=False))
    


tensor([[0.0427, 0.7091, 0.0296]], device='cuda:0')
0.04269259423017502 0.7090549468994141 0.029589863494038582


In [ ]:
frame = {'count': 0}
def save_data(image):
    
    processed_image = process_frame(image)
    img_name = f"{frame['count']:05d}.png"
    cv2.imwrite(os.path.join(IMG_RGB, img_name), image)
    cv2.imwrite(os.path.join(IMG_SEG, img_name), processed_image)

    expert_control = expert_vehicle.get_control()
    throttle = expert_control.throttle
    steer = expert_control.steer
    brake = expert_control.brake
    
    dagger_log.append({
        'img': img_name,
        'steer': steer,
        'throttle': throttle,
        'brake': brake,
    })

    frame['count'] += 1
    if frame['count'] >= 10000:
        camera_sensor.stop()

In [ ]:
traffic_manager.ignore_lights_percentage(vehicle, 100.0)
traffic_manager.ignore_signs_percentage(vehicle, 100.0)

vehicle.set_autopilot(True)

for i in range(10000):
    try:
        save_data(image)
        time.sleep(0.1)
    except:
        pass
with open(os.path.join(logs, "logs.json"), "w") as f:
    json.dump(data_log, f, indent=2)
    print("Done logging")

True


In [ ]:
while True:
    processed_image = process_frame(image)
    
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break

    combined_image = np.hstack((processed_image, image))

    cv2.imshow("Camera", combined_image)
    cv2.waitKey(1)

cv2.destroyAllWindows()